In [5]:
# imports
import numpy as np  # Required version 1.19.2
import pandas as pd # Required version 1.1.3
import matplotlib.pyplot as plt
from tqdm import tqdm

from glob import glob
from nbc import NBC

In [6]:
"""
Predict win based on team and opponent team weapon and level
"""

# Read in dataset
dataset_dir = "./dataset/"
test_dataset_path = dataset_dir + "2020-11-21.csv"


used_columns = [
    "lobby-mode",
    "lobby",
    "mode",
    "stage",
    "win",

    "A1-weapon",
    "A1-rank",
    "A1-level",
    "A2-weapon",
    "A2-rank",
    "A2-level",
    "A3-weapon",
    "A3-rank",
    "A3-level",
    "A4-weapon",
    "A4-rank",
    "A4-level",

    "B1-weapon",
    "B1-rank",
    "B1-level",
    "B2-weapon",
    "B2-rank",
    "B2-level",
    "B3-weapon",
    "B3-rank",
    "B3-level",
    "B4-weapon",
    "B4-rank",
    "B4-level"
]

label_name = "win"


In [7]:
train_dataset.dtypes

lobby-mode     object
lobby          object
mode           object
stage          object
win            object
A1-weapon      object
A1-rank        object
A1-level        int64
A2-weapon      object
A2-rank        object
A2-level        int64
A3-weapon      object
A3-rank        object
A3-level        int64
A4-weapon      object
A4-rank        object
A4-level      float64
B1-weapon      object
B1-rank        object
B1-level        int64
B2-weapon      object
B2-rank        object
B2-level        int64
B3-weapon      object
B3-rank        object
B3-level        int64
B4-weapon      object
B4-rank        object
B4-level      float64
dtype: object

In [8]:
# Model config
stages = [
    'anchovy',
    'arowana',
    'bbass',
    'mongara',
    'ajifry',
    'kombu',
    'ama',
    'mozuku',
    'zatou',
    'manta',
    'tachiuo',
    'fujitsubo',
    'otoro',
    'shottsuru',
    'hokke',
    'devon',
    'mutsugoro',
    'engawa',
    'gangaze',
    'chozame',
    'battera',
    'sumeshi',
    'hakofugu',
    'mystery_11',
    'mystery_21',
    'mystery_07',
    'mystery_05',
    'mystery_18',
    'mystery_23',
    'mystery_03',
    'mystery_09',
    'mystery_16',
    'mystery_24',
    'mystery_15',
    'mystery',
    'mystery_12',
    'mystery_14',
    'mystery_08',
    'mystery_22',
    'mystery_20',
    'mystery_17',
    'mystery_02',
    'mystery_19',
    'mystery_13',
    'mystery_01',
    'mystery_04',
    'mystery_10',
    'mystery_06'
]

weapons = [
    '52gal',
    '52gal_becchu',
    '52gal_deco',
    '96gal',
    '96gal_deco',
    'bold',
    'bold_7',
    'bold_neo',
    'bottlegeyser',
    'bottlegeyser_foil',
    'heroshooter_replica',
    'jetsweeper',
    'jetsweeper_custom',
    'momiji',
    'nzap83',
    'nzap85',
    'nzap89',
    'ochiba',
    'octoshooter_replica',
    'prime',
    'prime_becchu',
    'prime_collabo',
    'promodeler_mg',
    'promodeler_pg',
    'promodeler_rg',
    'sharp',
    'sharp_neo',
    'sshooter',
    'sshooter_becchu',
    'sshooter_collabo',
    'wakaba',
    'clashblaster',
    'clashblaster_neo',
    'heroblaster_replica',
    'hotblaster',
    'hotblaster_custom',
    'longblaster',
    'longblaster_custom',
    'longblaster_necro',
    'nova',
    'nova_becchu',
    'nova_neo',
    'rapid',
    'rapid_becchu',
    'rapid_deco',
    'rapid_elite',
    'rapid_elite_deco',
    'h3reelgun',
    'h3reelgun_cherry',
    'h3reelgun_d',
    'l3reelgun',
    'l3reelgun_becchu',
    'l3reelgun_d',
    'dualsweeper',
    'dualsweeper_custom',
    'heromaneuver_replica',
    'kelvin525',
    'kelvin525_becchu',
    'kelvin525_deco',
    'maneuver',
    'maneuver_becchu',
    'maneuver_collabo',
    'quadhopper_black',
    'quadhopper_white',
    'sputtery',
    'sputtery_clear',
    'sputtery_hue',
    'carbon',
    'carbon_deco',
    'dynamo',
    'dynamo_becchu',
    'dynamo_tesla',
    'heroroller_replica',
    'splatroller',
    'splatroller_becchu',
    'splatroller_collabo',
    'variableroller',
    'variableroller_foil',
    'herobrush_replica',
    'hokusai',
    'hokusai_becchu',
    'hokusai_hue',
    'pablo',
    'pablo_hue',
    'pablo_permanent',
    'bamboo14mk1',
    'bamboo14mk2',
    'bamboo14mk3',
    'herocharger_replica',
    'liter4k',
    'liter4k_custom',
    'liter4k_scope',
    'liter4k_scope_custom',
    'soytuber',
    'soytuber_custom',
    'splatcharger',
    'splatcharger_becchu',
    'splatcharger_collabo',
    'splatscope',
    'splatscope_becchu',
    'splatscope_collabo',
    'squiclean_a',
    'squiclean_b',
    'squiclean_g',
    'bucketslosher',
    'bucketslosher_deco',
    'bucketslosher_soda',
    'explosher',
    'explosher_custom',
    'furo',
    'furo_deco',
    'heroslosher_replica',
    'hissen',
    'hissen_hue',
    'screwslosher',
    'screwslosher_becchu',
    'screwslosher_neo',
    'barrelspinner',
    'barrelspinner_deco',
    'barrelspinner_remix',
    'herospinner_replica',
    'hydra',
    'hydra_custom',
    'kugelschreiber',
    'kugelschreiber_hue',
    'nautilus47',
    'nautilus79',
    'splatspinner',
    'splatspinner_becchu',
    'splatspinner_collabo',
    'campingshelter',
    'campingshelter_camo',
    'campingshelter_sorella',
    'heroshelter_replica',
    'parashelter',
    'parashelter_sorella',
    'spygadget',
    'spygadget_becchu',
    'spygadget_sorella'
 ]

ranks = [
    "c-",
    "c",
    "c+",
    "b-",
    "b",
    "b+",
    "a-",
    "a",
    "a+",
    "s",
    "s+",
    "x"
]

In [9]:
model_params = {
    "smoothing_params": {
        "lobby-mode": ["gachi", "regular", "fest"],
        "lobby": ["standard", "squad_2", "squad_4", "private"],
        "mode": ["nawabari", "area", "yagura", "hoko", "asari"],
        "stage": stages,
        "A1-weapon": weapons,
        "A1-rank": ranks,
        "A2-weapon": weapons,
        "A2-rank": ranks,
        "A3-weapon": weapons,
        "A3-rank": ranks,
        "A4-weapon": weapons,
        "A4-rank": ranks,
        "B1-weapon": weapons,
        "B1-rank": ranks,
        "B2-weapon": weapons,
        "B2-rank": ranks,
        "B3-weapon": weapons,
        "B3-rank": ranks,
        "B4-weapon": weapons,
        "B4-rank": ranks
    },
    "continuous": {
        "A1-level": True,
        "A2-level": True,
        "A3-level": True,
        "A4-level": True,
        "B1-level": True,
        "B2-level": True,
        "B3-level": True,
        "B4-level": True
    },
    "max_discretize_count": 10, 
    "sample_frac": 0.5, 
    "partition": "qcut",
    "q":10,
    "smoothing": True
}

In [22]:
# Create model
nbc = NBC(model_params)

In [23]:
# Train on model
training_files = glob(dataset_dir + "2020-11-*.csv")[10:]
training_files = tqdm(training_files)
for training_file_path in training_files:
    training_files.set_description(f"Processing {training_file_path}")
    train_dataset = pd.read_csv(training_file_path, engine="python", usecols=used_columns)
    # Drop NA (Use Ranked and League battle data only for now)
    train_dataset = train_dataset.dropna()
    label_name = "win"

    nbc.train(train_dataset, label_name=label_name)

Processing ./dataset/2020-11-14.csv: 100%|██████████| 12/12 [00:02<00:00,  5.55it/s]


In [26]:
alpha_portion = nbc.priors["alpha"] / nbc.priors["total_size"]
bravo_portion = nbc.priors["bravo"] / nbc.priors["total_size"]
print(f"Alpha percent: {alpha_portion}\tBravo percent: {bravo_portion}")

Alpha percent: 0.5333434450680015	Bravo percent: 0.4666565549319986


In [25]:
# Evaluate model
testing_files = glob(dataset_dir + "2020-11-*.csv")
testing_files = tqdm(testing_files)
for testing_file_path in testing_files:
    testing_files.set_description(f"Evaluating {testing_file_path}")
    test_dataset = pd.read_csv(testing_file_path, engine="python", usecols=used_columns)
    # Drop NA (Use Ranked and League battle data only for now)
    test_dataset = test_dataset.dropna()
    label_name = "win"
    nbc.evaluate(test_dataset, label_name)

Evaluating ./dataset/2020-11-18.csv:   5%|▍         | 1/22 [00:09<03:23,  9.68s/it]ZERO-ONE LOSS=0.4603
SQUARED LOSS=0.2675 Test Accuracy=0.5397
Evaluating ./dataset/2020-11-22.csv:   9%|▉         | 2/22 [00:19<03:15,  9.78s/it]ZERO-ONE LOSS=0.4462
SQUARED LOSS=0.2560 Test Accuracy=0.5538
Evaluating ./dataset/2020-11-20.csv:  14%|█▎        | 3/22 [00:36<03:43, 11.77s/it]ZERO-ONE LOSS=0.4512
SQUARED LOSS=0.2614 Test Accuracy=0.5488
Evaluating ./dataset/2020-11-08.csv:  18%|█▊        | 4/22 [00:46<03:23, 11.28s/it]ZERO-ONE LOSS=0.4781
SQUARED LOSS=0.2705 Test Accuracy=0.5219
Evaluating ./dataset/2020-11-09.csv:  23%|██▎       | 5/22 [00:57<03:13, 11.39s/it]ZERO-ONE LOSS=0.4687
SQUARED LOSS=0.2680 Test Accuracy=0.5313
Evaluating ./dataset/2020-11-21.csv:  27%|██▋       | 6/22 [01:06<02:48, 10.53s/it]ZERO-ONE LOSS=0.4712
SQUARED LOSS=0.2658 Test Accuracy=0.5288
Evaluating ./dataset/2020-11-10.csv:  32%|███▏      | 7/22 [01:21<02:56, 11.79s/it]ZERO-ONE LOSS=0.4486
SQUARED LOSS=0.2613 Test A

ZeroDivisionError: division by zero

In [27]:
sample = {
    "lobby-mode": "gachi",
    "lobby": "squad_2",
    "mode": "area",
    "stage": "zatou",

    "A1-weapon": "nzap85",
    "A1-rank": "x",
    "A1-level": 100,
    "A2-weapon": "dynamo_becchu",
    "A2-rank": "x",
    "A2-level": 102,
    "A3-weapon": "heroblaster_replica",
    "A3-rank": "x",
    "A3-level": 89,
    "A4-weapon": "bucketslosher_soda",
    "A4-rank": "x",
    "A4-level": 112,

    "B1-weapon": "nzap85",
    "B1-rank": "x",
    "B1-level": 33,
    "B2-weapon": "carbon_deco",
    "B2-rank": "x",
    "B2-level": 36,
    "B3-weapon": "prime_becchu",
    "B3-rank": "x",
    "B3-level": 33,
    "B4-weapon": "prime_becchu",
    "B4-rank": "x",
    "B4-level": 59
}

In [28]:
nbc.predict(sample)

('alpha', 0.6750506029272288)